In [ ]:
# 1. IMPORT LIBRARIES
import requests
import json  
import csv
import pandas as pd

In [4]:
# 2.2 USE Records API 1.0 TO RETRIEVE DATA
# dataset2: "inscritos-em-lic-dentro-do-tmrg-180-dias@spms"  # dataset identifier

BASE_URL = "https://transparencia.sns.gov.pt/api/records/1.0/search/"
DATASET_ID = "inscritos-em-lic-dentro-do-tmrg-180-dias@spms"  # dataset identifier
ROWS = 1000  # number of rows per request (you can adjust)
start = 0
all_records = []

while True:
    params = {
        "dataset": DATASET_ID,
        "rows": ROWS,
        "start": start
    }
    response = requests.get(BASE_URL, params=params)
    response.raise_for_status()
    data = response.json()
    
    records = data.get("records", [])
    if not records:
        break
    
    all_records.extend(records)
    print(f"Fetched {len(records)} records, total so far: {len(all_records)}")
    
    start += ROWS

print("Total records fetched:", len(all_records))

# 3.2 SAVE TO CSV
if all_records:
    fieldnames = all_records[0]["fields"].keys()
    with open("inscritos_lic_tmrg_180_api.csv", "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for rec in all_records:
            writer.writerow(rec["fields"])

print("Saved to inscritos_lic_tmrg_180_api.csv")


Fetched 1000 records, total so far: 1000
Fetched 1000 records, total so far: 2000
Fetched 1000 records, total so far: 3000
Fetched 1000 records, total so far: 4000
Fetched 215 records, total so far: 4215
Total records fetched: 4215
Saved to inscritos_lic_tmrg_180_api.csv


In [ ]:
# 4. LOADS THE CSV INTO A DATAFRAME
df_inscritos = pd.read_csv("inscritos_lic_tmrg_180_api.csv")

# 5.1 CHECK BASIC INFO about the dataset
print("=== Dataset Info ===")
print(df_inscritos.info()) 

=== Dataset Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4215 entries, 0 to 4214
Data columns (total 7 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   tempo                                         4215 non-null   object 
 1   instituicao                                   4215 non-null   object 
 2   localizacao_geografica                        4215 non-null   object 
 3   no_de_doentes_inscritos_sigic                 4215 non-null   int64  
 4   de_inscritos_em_lic_dentro_do_tmrg            4215 non-null   float64
 5   no_de_doentes_inscritos_dentro_do_tmrg_sigic  4215 non-null   int64  
 6   regiao                                        4215 non-null   object 
dtypes: float64(1), int64(2), object(4)
memory usage: 230.6+ KB
None


In [4]:
# 5.2 Check dataset shape (rows, columns)
print("\n=== Shape ===")
print(df_inscritos.shape)


=== Shape ===
(4215, 7)


In [5]:
# 5.3 Check column names
print("\n=== Column Names ===")
print(df_inscritos.columns.tolist())


=== Column Names ===
['tempo', 'instituicao', 'localizacao_geografica', 'no_de_doentes_inscritos_sigic', 'de_inscritos_em_lic_dentro_do_tmrg', 'no_de_doentes_inscritos_dentro_do_tmrg_sigic', 'regiao']


In [6]:
# 5.4 Count each data type
print(df_inscritos.dtypes.value_counts())


object     4
int64      2
float64    1
Name: count, dtype: int64


In [7]:
# 5.5 Check data types of each column
print("\n=== Data Types ===")
print(df_inscritos.dtypes)


=== Data Types ===
tempo                                            object
instituicao                                      object
localizacao_geografica                           object
no_de_doentes_inscritos_sigic                     int64
de_inscritos_em_lic_dentro_do_tmrg              float64
no_de_doentes_inscritos_dentro_do_tmrg_sigic      int64
regiao                                           object
dtype: object


In [8]:
# 5.6 Preview the first few rows
print("\n=== Head of Dataset ===")
print(df_inscritos.head())


=== Head of Dataset ===
     tempo                              instituicao    localizacao_geografica  \
0  2018-01         Centro Hospitalar de Leiria, EPE  [39.7414812, -8.7982706]   
1  2018-01            Hospital Dr. Francisco Zagalo  [40.8571967, -8.6318023]   
2  2018-01    Unidade Local de Saúde da Guarda, EPE  [40.5309916, -7.2734793]   
3  2018-01  Centro Hospitalar Barreiro/Montijo, EPE  [38.6554005, -9.0592392]   
4  2018-01        Centro Hospitalar Médio Tejo, EPE   [39.478072, -8.5404429]   

   no_de_doentes_inscritos_sigic  de_inscritos_em_lic_dentro_do_tmrg  \
0                           6500                                82.7   
1                            414                                80.0   
2                           2023                                61.8   
3                           2811                                60.9   
4                           4159                                71.7   

   no_de_doentes_inscritos_dentro_do_tmrg_sigic        

In [9]:
# 6.1 CHECK FOR MISSING VALUES
print("\n=== Missing Values ===")
print(df_inscritos.isnull().sum())


=== Missing Values ===
tempo                                           0
instituicao                                     0
localizacao_geografica                          0
no_de_doentes_inscritos_sigic                   0
de_inscritos_em_lic_dentro_do_tmrg              0
no_de_doentes_inscritos_dentro_do_tmrg_sigic    0
regiao                                          0
dtype: int64


In [ ]:
# 6.2 Filter: remove hospitals that do not offer the full range of surgeries, as they are not informative for testing the study hypothesis

# List of hospitals to remove
excluded_hospitals = [
    "Hospital Dr. Francisco Zagalo",
    "Instituto Gama Pinto",
    "Hospital Arcebispo João Crisóstomo",
    "Hospital José Luciano de Castro",
    'Instituto Português Oncologia de Coimbra, EPE',
]

# Filter out excluded hospitals (df_filtered is a filtered view)
df_filtered = df_inscritos[~df_inscritos["instituicao"].isin(excluded_hospitals)]

In [12]:
# 6.3 After cleaning, there are no missing values
print("\n=== Missing Values ===")
print(df_filtered.isnull().sum())


=== Missing Values ===
tempo                                           0
instituicao                                     0
localizacao_geografica                          0
no_de_doentes_inscritos_sigic                   0
de_inscritos_em_lic_dentro_do_tmrg              0
no_de_doentes_inscritos_dentro_do_tmrg_sigic    0
regiao                                          0
dtype: int64


In [13]:
# 6.4 Verify cleaned dataset shape (rows, columns)
print("\n=== Shape ===")
print(df_filtered.shape)


=== Shape ===
(3927, 7)


In [14]:
# 7. DATA TYPE CONVERSION is not necessary, type is fine as is


In [15]:
# 8.1 CHECK UNIQUE VALUES

# DEFINE CATEGORICAL VAR WITH A LIST
categorical_vars = df_filtered.select_dtypes(include=['object']).columns.tolist()
# Count unique values in each categorical var
print("\n=== Unique Values per Column ===")
print(f"{categorical_vars}: {df_filtered[categorical_vars].nunique()} unique values")


=== Unique Values per Column ===
['tempo', 'instituicao', 'localizacao_geografica', 'regiao']: tempo                     93
instituicao               83
localizacao_geografica    85
regiao                     5
dtype: int64 unique values


In [16]:
# 8.2 Check unique values in each categorical var
for col in categorical_vars:
    print(col, "→", df_filtered[col].unique())

tempo → ['2018-01' '2018-02' '2018-03' '2018-04' '2018-05' '2018-06' '2018-07'
 '2018-08' '2018-09' '2018-10' '2018-11' '2018-12' '2019-01' '2019-02'
 '2019-03' '2019-04' '2019-05' '2019-06' '2019-07' '2019-08' '2019-09'
 '2019-10' '2019-11' '2019-12' '2020-01' '2020-02' '2020-03' '2020-04'
 '2020-05' '2020-06' '2020-07' '2020-08' '2020-09' '2020-10' '2020-11'
 '2020-12' '2021-01' '2021-02' '2021-03' '2021-04' '2021-05' '2021-06'
 '2021-07' '2021-08' '2021-09' '2021-10' '2021-11' '2021-12' '2022-01'
 '2022-02' '2022-03' '2022-04' '2022-05' '2022-06' '2022-07' '2022-08'
 '2022-09' '2022-10' '2022-11' '2022-12' '2023-01' '2023-02' '2023-03'
 '2023-04' '2023-05' '2023-06' '2023-07' '2023-08' '2023-09' '2023-10'
 '2023-11' '2023-12' '2024-01' '2025-01' '2024-02' '2025-02' '2024-03'
 '2025-03' '2024-04' '2025-04' '2024-05' '2025-05' '2024-06' '2025-06'
 '2024-07' '2025-07' '2024-08' '2025-08' '2024-09' '2025-09' '2024-10'
 '2024-11' '2024-12']
instituicao → ['Centro Hospitalar de Leiria, EP

In [ ]:
# 9. standardise column names to English and snake_case
df_filtered_v2 = df_filtered.copy()  
df_filtered_v2 = df_filtered_v2.rename(columns={
    "tempo": "period",
    "regiao": "region",
    "instituição": "health_centre",
    "localizacao_geografica": "location",
    "no_de_doentes_inscritos_sigic": "tot_patients_waiting",
    "no_de_doentes_inscritos_dentro_do_tmrg_sigic": "patients_normal_waiting_time",
    "de_inscritos_em_lic_dentro_do_tmrg": "pct_patients_normal_waiting_time"
})
 
print(df_filtered_v2.columns)

Index(['period', 'instituicao', 'location', 'tot_patients_waiting',
       'pct_patients_normal_waiting_time', 'patients_normal_waiting_time',
       'region'],
      dtype='object')


In [26]:
df_filtered_v2['region'] = df_filtered_v2['region'].replace({
    "Região de Saúde Norte": "Northern",
    "Região de Saúde LVT": "Lisbon and Tagus Valley",
    "Região de Saúde do Centro": "Central",
    "Região de Saúde do Alentejo": "Alentejo",
    "Região de Saúde do Algarve": "Algarve"
})
df_filtered_v2['region'].unique()


array(['Central', 'Lisbon and Tagus Valley', 'Northern', 'Alentejo',
       'Algarve'], dtype=object)

In [ ]:
# 10. Save updated dataset
df_filtered_v2.to_csv("inscritos_cleaned_en.csv", index=False)  
df_filtered_v2.to_excel("inscritos_cleaned_en.xlsx", index=False)

In [28]:
# 11. FREQUENCIES
# List of surgery columns
inscritos_cols = [
    "tot_patients_waiting",
    "patients_normal_waiting_time",
]

# Group by region and sum
region_summary = df_filtered_v2.groupby("region")[inscritos_cols].sum()
region_summary["total_inscritos"] = region_summary.sum(axis=1)

# Total surgeries across all regions
totals = region_summary.sum()

# Add percentage columns
for col in inscritos_cols:
    region_summary[col + "_pct"] = (region_summary[col] / totals[col] * 100).round(1)
region_summary["total_pct"] = (region_summary["total_inscritos"] / totals["total_inscritos"] * 100).round(1)

# Add Total row
total_row = pd.DataFrame(region_summary.sum()).T
total_row.index = ["Total"]
for col in inscritos_cols:
    total_row[col + "_pct"] = 100.0
total_row["total_pct"] = 100.0
region_summary = pd.concat([region_summary, total_row])

# Sort by total inscritos (Total row at bottom)
sorted_regions = region_summary.drop("Total").sort_values("total_inscritos", ascending=False).index
region_summary = region_summary.loc[list(sorted_regions) + ["Total"]]

# Format numbers and percentages
formatted = region_summary.copy()

# Format numbers with thousands separator
for col in inscritos_cols + ["total_inscritos"]:
    formatted[col] = formatted[col].apply(lambda x: f"{int(x):,}")

# Format percentages with 1 decimal + %
pct_cols = [c + "_pct" for c in inscritos_cols] + ["total_pct"]
for col in pct_cols:
    formatted[col] = formatted[col].apply(lambda x: f"{x:.1f}%")

# Move region column first
formatted.insert(0, "Region", formatted.index)

# Optional: reorder columns for better presentation
display_cols = ["Region"]
for col in inscritos_cols:
    display_cols.extend([col, col + "_pct"])
display_cols.extend(["total_inscritos", "total_pct"])
formatted = formatted[display_cols]

# Display table
print(formatted)


                                          Region tot_patients_waiting  \
Northern                                Northern            7,841,097   
Lisbon and Tagus Valley  Lisbon and Tagus Valley            8,153,668   
Central                                  Central            4,005,810   
Alentejo                                Alentejo              830,788   
Algarve                                  Algarve              921,234   
Total                                      Total           21,752,597   

                        tot_patients_waiting_pct patients_normal_waiting_time  \
Northern                                   36.0%                    6,286,471   
Lisbon and Tagus Valley                    37.5%                    4,836,770   
Central                                    18.4%                    2,769,101   
Alentejo                                    3.8%                      586,398   
Algarve                                     4.2%                      472,221   
To